<img style="float: left;" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Personas

Empower multi-agent systems where specialists are defined to execute requests they were designed to fulfill.  

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [11]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"

using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// Retrieve the configuration settings for the Azure OpenAI service
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];

Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0

Personas leverage the Chat Completion service from the Semantic Kernel Agents package.  

In [12]:
// Import the Semantic Kernel NuGet package
#r "nuget: Microsoft.SemanticKernel, 1.11.1"
#r "nuget: Microsoft.SemanticKernel.Experimental.Agents, 1.11.1-alpha"  

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Experimental.Agents;

var name = "Dwight Eisenhower";
var instructions = """
You are a decision intelligence strategist with the knowledge of Dwight Eisenhower's Decision Prioritization Matrix. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";
var description = "A chat bot that replies to the message with recommending the Eisenhower Decision Priorotization Matrix.";

#pragma warning disable SKEXP0101
var agent = await new AgentBuilder()
    .WithAzureOpenAIChatCompletion(
        model: azureOpenAIModelDeploymentName,
        endpoint: azureOpenAIEndpoint,
        apiKey: azureOpenAIAPIKey)
    .WithInstructions(instructions)
    .WithName(name)
    .WithDescription(description)
    .BuildAsync();
#pragma warning restore SKEXP0101

Installed Packages Microsoft.SemanticKernel, 1.11.1 Microsoft.SemanticKernel.Experimental.Agents, 1.11.1-alpha

In [ ]:
#pragma warning disable SKEXP0101
var thread = await agent.NewThreadAsync();

string[] messages = [
    "How do you make decisions?",
    "How should I priotize a lazy day?",
    "How does your favorite decision framework compare to other decision frameworks?"
];

foreach (var response in messages.Select(m => thread.InvokeAsync(agent, m)))
{
    await foreach (var message in response)
    {
        string speaker = message.Role == "user" ? "You" : "Dwight Eisenhower";
        Console.WriteLine($"# {speaker}: {message.Content}");
        Console.WriteLine();
        Console.WriteLine();
    }
}

// Delete the assistants and threads
await thread.DeleteAsync();
await agent.DeleteAsync();
#pragma warning restore SKEXP0101

# You: How do you make decisions?


